In [ ]:
import numpy as np
from torch import nn
from torch import optim
from torch import FloatTensor

In [ ]:
# cargamos los datos
data_trn = np.loadtxt("XOR_trn.csv", delimiter=",")
data_tst = np.loadtxt("XOR_tst.csv", delimiter=",")

In [ ]:
# datos como tensores de pytorch
X_train_tensor = FloatTensor(data_trn[:,:-1])
y_train_tensor = FloatTensor(data_trn[:,-1])

X_test_tensor = FloatTensor(data_tst[:,:-1])
y_test = data_tst[:,-1]

In [ ]:
# definición del modelo
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()

        self.capa1 = nn.Linear(2, 2)
        self.capa2 = nn.Linear(2, 1)

        self.activacion = nn.Tanh()

    def forward(self, x):
        y = self.capa1(x)
        y = self.activacion(y)

        y = self.capa2(y)
        y = self.activacion(y)

        return y

In [ ]:
# entrenamiento
mlp_pyt = MLP()                                      # instanciación del modelo
loss_criterion = nn.MSELoss()                        # criterio de error cuadrático medio
optimizer = optim.SGD(mlp_pyt.parameters(), lr=0.1, momentum=0.9)  # método de gradiente y velocidad de aprendizaje

mlp_pyt.train()
for epoch in range(1024):
    optimizer.zero_grad()                            # inicializacion de gradientes
    y_pred = mlp_pyt(X_train_tensor)                 # pasada hacia adelante
    loss = loss_criterion(y_pred.squeeze(), y_train_tensor) # se miden los erroes
    loss.backward()                                  # pasada hacia atrás
    optimizer.step()                                 # ajuste de pesos

In [ ]:
# prueba
mlp_pyt.eval()
y_test_pred = mlp_pyt(X_test_tensor)          # pasada hacia adelante
y_test_pred = y_test_pred.detach().round()    # redondea a +1/-1
acc_pyt = sum(y_test_pred.squeeze().numpy() == y_test) / len(y_test) # exactitud

print(acc_pyt)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(1, 1, figsize=(10,6))

# CLASIFICACION
C = []
for i in range(len(y_test)):

    if (np.sign(y_test[i]) == np.sign(y_test_pred[i])) and (np.sign(y_test[i]) == 1):
        C.append('blue')
    if (np.sign(y_test[i]) == np.sign(y_test_pred[i])) and (np.sign(y_test[i]) == -1):
        C.append('red')
    if (np.sign(y_test[i]) != np.sign(y_test_pred[i])) and (np.sign(y_test[i]) == 1):
        C.append('cyan')
    if (np.sign(y_test[i]) != np.sign(y_test_pred[i])) and (np.sign(y_test[i]) == -1):
        C.append('magenta')

ax.scatter(X_test_tensor[:,0], X_test_tensor[:,1], 20, C)
ax.set_xlim(-1.1,1.1)
ax.set_ylim(-1.1,1.1)
ax.set_xlabel('X1', fontsize=14)
ax.set_ylabel('X2', fontsize=14)
ax.grid(True)

# CONSTRUCCION DE LA FRONTERA DE DECISION
x = torch.tensor([-1.5, 1.5])
W1 = mlp_pyt.capa1.weight[0,:].flatten().detach().cpu()
B1 = mlp_pyt.capa1.bias[0].detach().cpu()
b1 = -B1/W1[1]
m1 = W1[0]/W1[1]

W2 = mlp_pyt.capa1.weight[1,:].flatten().detach().cpu()
B2 = mlp_pyt.capa1.bias[1].detach().cpu()
b2 = -B2/W2[1]
m2 = W2[0]/W2[1]

ax.plot(x, b1 - m1 * x, ':k', lw=2)
ax.plot(x, b2 - m2 * x, ':k', lw=2)
ax.set_xlim(-1.1,1.1)
ax.set_ylim(-1.1,1.1)
ax.grid(True)